### LOGISTIC REGRESSION
In this assignment we try to find a relation between average weight of granules and total surface area to see if a material is viable as a catalyst

In [1]:
# Importing libraries here
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Feature mapping
Sometimes, we are given an inadequate number of features for which training the dataset becomes difficult.  
Hence we create new features of by taking polynomial products of existing ones.

In [2]:
def feature_map(points):
    points = np.array(points)
    x, y = points[:, 0], points[:, 1]
    
    features = []
    
    # degree 0
    features.append(np.ones_like(x))      # 1
    
    # degree 1
    features.append(x)                     # x
    features.append(y)                     # y
    
    # degree 2
    features.append(x**2)                  # x^2
    features.append(x*y)                   # xy
    features.append(y**2)                  # y^2
    
    # degree 3
    features.append(x**3)                  # x^3
    features.append(x**2 * y)              # x^2y
    features.append(x * y**2)              # xy^2
    features.append(y**3)                  # y^3
    
    # degree 4
    features.append(x**4)                  # x^4
    features.append(x**3 * y)              # x^3y
    features.append(x**2 * y**2)            # x^2y^2
    features.append(x * y**3)              # xy^3
    features.append(y**4)                  # y^4
    
    return np.column_stack(features)


### Creating the class for Logistic Regression

In [5]:
class LogisticRegression:

    def __init__(self) -> None:
        self.weights: np.ndarray | None = None
        self.bias: float | None = None


    # Sigmoid function
    ### TODO 2
    def __sigmoid(self, z: np.ndarray) -> np.ndarray:
        return 1 / (1 + np.exp(-z))


    # Returns probabilities of being true
    ### TODO 3
    def predict_probability(self, X: np.ndarray) -> np.ndarray:
        z = np.dot(X, self.weights) + self.bias
        return self.__sigmoid(z)


    # Returns true/false (based on the probabilities)
    ### TODO 4
    def predict(self, X: np.ndarray) -> np.ndarray:
        probs = self.predict_probability(X)
        return (probs >= 0.5).astype(int)


    # Returns loss, gradient wrt weights (dw), gradient wrt bias (db)
    ### TODO 5 
    def __loss(self, X: np.ndarray, y: np.ndarray, lambda_reg: float = 0) -> tuple:
        m = X.shape[0]

        y_pred = self.predict_probability(X)

        # Binary cross-entropy loss
        loss = (-1 / m) * np.sum(
            y * np.log(y_pred + 1e-9) + (1 - y) * np.log(1 - y_pred + 1e-9)
        )

        # Gradients
        dw = (1 / m) * np.dot(X.T, (y_pred - y))
        db = (1 / m) * np.sum(y_pred - y)

        # L2 Regularization
        loss += (lambda_reg / (2 * m)) * np.sum(self.weights ** 2)
        dw += (lambda_reg / m) * self.weights

        return loss, dw, db


    # Training using gradient descent
    ### TODO 6
    def fit(self, X: np.ndarray, y: np.ndarray, epochs: int = 500,
            learning_rate: float = 0.01, threshold: float = 0.0001, 
            lambda_reg: float = 1) -> None:

        n_features = X.shape[1]

        # Initialize weights and bias
        self.weights = np.zeros(n_features)
        self.bias = 0.0

        prev_loss = float('inf')

        for _ in range(epochs):
            loss, dw, db = self.__loss(X, y, lambda_reg)

            # Gradient descent update
            self.weights -= learning_rate * dw
            self.bias -= learning_rate * db

            # Early stopping
            if abs(prev_loss - loss) < threshold:
                break

            prev_loss = loss


In [3]:
# Importing data
df = pd.read_csv('logistic_data.csv')
data = df.to_numpy()
X = data[:, :2]
y = data[:, 2]

In [6]:
# Creating train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
def z_score(X: np.ndarray) -> tuple:
    # Mean and standard deviation per feature (column-wise)
    x_mean = np.mean(X, axis=0)
    x_std = np.std(X, axis=0)

    # Avoid division by zero
    x_std = np.where(x_std == 0, 1, x_std)

    # Z-score normalization
    x = (X - x_mean) / x_std

    return x, x_mean, x_std


In [8]:
# Normalizing the data (we use the same constants to maintain consistency)
X_train, x_mean, x_std = z_score(X_train)
X_test = (X_test - x_mean) / x_std
x_train = feature_map(X_train)
x_test = feature_map(X_test)

In [9]:
# Visualizing how the boundary curve looks like
def plot_decision_boundary(X_original, y, model, resolution=500):
    # Set up the grid for the decision boundary
    x_min, x_max = X_original[:, 0].min() - 1, X_original[:, 0].max() + 1
    y_min, y_max = X_original[:, 1].min() - 1, X_original[:, 1].max() + 1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, resolution),
                         np.linspace(y_min, y_max, resolution))
    
    # Flatten the grid points and map to expanded features
    grid_original = np.c_[xx.ravel(), yy.ravel()]
    grid_expanded = feature_map(grid_original)
    
    # Predict the grid values for decision boundary
    Z = model.predict(grid_expanded)
    Z = Z.reshape(xx.shape)
    
    # Plot the data points
    true_points = X_original[y == 1]
    false_points = X_original[y == 0]
    plt.scatter(true_points[:, 0], true_points[:, 1], label="True", c="blue", marker="o", s=20)
    plt.scatter(false_points[:, 0], false_points[:, 1], label="False", c="red", marker="x", s=20)

    # Plot the decision boundary
    plt.contour(xx, yy, Z, levels=[0.5], colors="black", linewidths=2)
    
    # Labeling and title
    plt.xlabel("Feature 1")
    plt.ylabel("Feature 2")
    plt.title("Decision Boundary and Data Points")
    plt.show()

We plot the decision boundary that the model predicts. This can be used to check for overfitting.  
If the boundary starts looking like an ameoba trying to fit every point, then it is a sign of overfitting.

In [11]:
model = LogisticRegression()
model.fit(
    x_train,
    y_train,
    epochs=1000,
    learning_rate=0.05,
    threshold=1e-5,
    lambda_reg=0.1
)

y_pred = model.predict(x_test)

accuracy = np.mean(y_pred == y_test) * 100
print(f"Your model has an accuracy of {accuracy}%")


Your model has an accuracy of 87.5%
